In [142]:
import glob
import os
import subprocess

highlevel_dirs = sum([
    #["../tools"], 
    #sorted(glob.glob("../sem16*")),
    sorted(glob.glob("../sem1*")),
], [])

print("Highlevel dirs:", highlevel_dirs)

Highlevel dirs: ['../sem10-file-attributes', '../sem11-mmap-instrumentation', '../sem12-fork-exec-pipe', '../sem13-signal', '../sem14-fifo-proc', '../sem15-ptrace', '../sem16-fcntl-dup-pipe', '../sem17-sockets-tcp-udp', '../sem18-multiplexing']


In [143]:

def convert(n, d):
    get_ipython().system("jupyter nbconvert {} --to markdown --output {}".format(n, d))
    #subprocess.check_call(["jupyter", "nbconvert", n, "--to", "markdown", "--output", d])

for subdir in highlevel_dirs:
    notebooks = glob.glob(subdir + "/*.ipynb")
    print(subdir, notebooks)
    for m in glob.glob(subdir + "/*.md"):
        os.remove(m)
    if len(notebooks) == 1:
        convert(notebooks[0], "README")
    else:
        for n in notebooks:
            convert(n, os.path.basename(n.replace(".ipynb", "")))
        

../sem10-file-attributes ['../sem10-file-attributes/file-attrib.ipynb']
[NbConvertApp] Converting notebook ../sem10-file-attributes/file-attrib.ipynb to markdown
[NbConvertApp] Writing 16319 bytes to ../sem10-file-attributes/README.md
../sem11-mmap-instrumentation ['../sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb']
[NbConvertApp] Converting notebook ../sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb to markdown
[NbConvertApp] Writing 23442 bytes to ../sem11-mmap-instrumentation/README.md
../sem12-fork-exec-pipe ['../sem12-fork-exec-pipe/fork-pipe-exec.ipynb']
[NbConvertApp] Converting notebook ../sem12-fork-exec-pipe/fork-pipe-exec.ipynb to markdown
[NbConvertApp] Writing 20684 bytes to ../sem12-fork-exec-pipe/README.md
../sem13-signal ['../sem13-signal/signal.ipynb']
[NbConvertApp] Converting notebook ../sem13-signal/signal.ipynb to markdown
[NbConvertApp] Writing 26542 bytes to ../sem13-signal/README.md
../sem14-fifo-proc ['../sem14-fifo-proc/fifo-proc.ipynb']


In [144]:
import re

def basic_improve(fname):
    with open(fname, "r") as f:
        r = f.read()
    for b in ["\x00", "\x1B", "\x08"]:
        r = r.replace(b, "")
    with open(fname, "w") as f:
        f.write(r)
    get_ipython().system("dos2unix {}".format(fname))

def improve_md(fname):
    with open(fname, "r") as f:
        r = f.read()
    r = r.replace("```python\n%%cpp", "```cpp\n%%cpp")
    r = r.replace('\n', "SUPER_SLASH" + "_N_REPLACER")
    r = re.sub(r'\<\!--MD_BEGIN_FILTER--\>.*?\<\!--MD_END_FILTER--\>', "", r)
    r = re.sub(r'(\#SET_UP_MAGIC_BEGIN.*?\#SET_UP_MAGIC_END)', "<too much code>", r)
    r = r.replace("SUPER_SLASH" + "_N_REPLACER", '\n')
    
    def file_repl(matchobj, path=os.path.dirname(fname)):
        fname = os.path.join(path, matchobj.group(1))
        if fname.find("__FILE__") == -1:
            with open(fname, "r") as f:
                return "\n```\n" + f.read() + "\n```\n"
    
    r = r.replace("</td>", "")
    r = r.replace("</tr>", "")
    
    r = re.sub(r'\<\!--MD_FROM_FILE (.*?) --\>', file_repl, r)
    with open(fname, "w") as f:
        f.write(r)
        
def improve_file(fname):
    basic_improve(fname)
    if fname.endswith(".md"):
        improve_md(fname)


In [145]:
for sfx in [".ipynb", ".md"]:
    for hdir in highlevel_dirs:
        for fname in glob.glob("./{}/*".format(hdir) + sfx):
            improve_file(fname)

dos2unix: converting file ./../sem10-file-attributes/file-attrib.ipynb to Unix format ...
dos2unix: converting file ./../sem11-mmap-instrumentation/mmap_and_instrumentation.ipynb to Unix format ...
dos2unix: converting file ./../sem12-fork-exec-pipe/fork-pipe-exec.ipynb to Unix format ...
dos2unix: converting file ./../sem13-signal/signal.ipynb to Unix format ...
dos2unix: converting file ./../sem14-fifo-proc/fifo-proc.ipynb to Unix format ...
dos2unix: converting file ./../sem15-ptrace/ptrace.ipynb to Unix format ...
dos2unix: converting file ./../sem16-fcntl-dup-pipe/fcntl-dup-pipe.ipynb to Unix format ...
dos2unix: converting file ./../sem17-sockets-tcp-udp/sockets-tcp-udp.ipynb to Unix format ...
dos2unix: converting file ./../sem18-multiplexing/epoll-poll-select-linuxaio.ipynb to Unix format ...
dos2unix: converting file ./../sem10-file-attributes/README.md to Unix format ...
dos2unix: converting file ./../sem11-mmap-instrumentation/README.md to Unix format ...
dos2unix: convertin

In [ ]:
cmds = []
add_cmd = "git add --ignore-errors "
add_cmd_f = "git add --ignore-errors -f "
for subdir in highlevel_dirs:
    for sfx in [".ipynb", ".md", ".c", ".cpp"]:
        cmds.append(add_cmd + " {}/*{}".format(subdir, sfx))
    cmds.append(add_cmd_f + " -f {}/bash_popen_tmp/*.html".format(subdir))
    cmds.append(add_cmd_f + " -f {}/interactive_launcher_tmp/*.log".format(subdir))
    
def execute_cmd(cmd):
    print(">", cmd)
    get_ipython().system(cmd)
    
for cmd in cmds:
    execute_cmd(cmd)
execute_cmd("git add -u")
execute_cmd("git commit -m 'yet another update'")
execute_cmd("git push origin master")

> git add --ignore-errors  ../sem10-file-attributes/*.ipynb
The file will have its original line endings in your working directory.
> git add --ignore-errors  ../sem10-file-attributes/*.md
The file will have its original line endings in your working directory.
> git add --ignore-errors  ../sem10-file-attributes/*.c
> git add --ignore-errors  ../sem10-file-attributes/*.cpp
fatal: pathspec '../sem10-file-attributes/*.cpp' did not match any files
> git add --ignore-errors -f  -f ../sem10-file-attributes/bash_popen_tmp/*.html
fatal: pathspec '../sem10-file-attributes/bash_popen_tmp/*.html' did not match any files
> git add --ignore-errors -f  -f ../sem10-file-attributes/interactive_launcher_tmp/*.log
fatal: pathspec '../sem10-file-attributes/interactive_launcher_tmp/*.log' did not match any files
> git add --ignore-errors  ../sem11-mmap-instrumentation/*.ipynb
The file will have its original line endings in your working directory.
> git add --ignore-errors  ../sem11-mmap-instrumentation/*.